## 두 파일 합쳐서 저장
{기업 : {년도 : 문장들모음리스트}}  
+  
{문장 : (e,s,g,p,n)} 점수  

In [ ]:
# pip install ipywidgets

In [1]:
import json
from tqdm.notebook import tqdm

# {기업 : {년도 : 문장들모음리스트}}
with open('total_kosdaq_result.json', 'r') as kosdaq_json:
    company_date_sentence = json.load(kosdaq_json)
# {문장 : (e,s,g,p,n)} 점수
with open('kosdaq_esgpn_data.json', 'r') as file:
    esgpn_scores = json.load(file)

for corp in tqdm(company_date_sentence):  # 기업으로 접근
    for date in company_date_sentence[corp]:  # 기업 -> 날짜로 접근
        sentences = company_date_sentence[corp][date]
        updated_sentences = []
        for sent in sentences:
            if sent in esgpn_scores:  # 문장이 ESGPN 점수를 가지고 있다면, 그 점수를 리스트에 저장
                updated_sentences.append([sent, esgpn_scores[sent]])
            else:
                updated_sentences.append([sent, (None, None, None, None, None)])  # 문장이 ESGPN 점수를 갖고 있지 않다면, None값으로 저장
        company_date_sentence[corp][date] = updated_sentences

# 파일로 저장 (크기가 큰 파일임)
with open('updated_total_kosdaq_result.json', 'w') as outfile:
    json.dump(company_date_sentence, outfile, ensure_ascii=False, indent=4)

  0%|          | 0/1717 [00:00<?, ?it/s]

## 파일 불러오기

In [2]:
# 데이터 불러오기
import json
with open('updated_total_kosdaq_result.json', 'r') as file:
    company_date_sentence = json.load(file)

## 키워드 추출

In [3]:
import pandas as pd
from collections import Counter
from konlpy.tag import Mecab

# Initialize Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

# 결과를 저장할 구조
result = {}

# 데이터 처리
for company, data in company_date_sentence.items():
    if '2023' in data:
        positive_keywords = []
        negative_keywords = []
        
        for sentence_data in data['2023']:
            sentence, scores = sentence_data
            E, S, G, P, N = scores
            
            # P, N이 None이 아니고, P와 N 점수 비교
            if P is not None and N is not None:
                # 문장에서 명사와 동사만 추출하여 키워드로 사용
                keywords = [token[0] for token in mecab.pos(sentence) if token[1][0] in ['N', 'V'] and len(token[0]) >= 2]
                
                if P > N:
                    positive_keywords.extend(keywords)
                elif P < N:
                    negative_keywords.extend(keywords)

        # 키워드 출현 빈도 계산 및 상위 10개 선택
        positive_counts = Counter(positive_keywords).most_common(10)
        negative_counts = Counter(negative_keywords).most_common(10)

        # 결과 저장
        result[company] = {'positive': positive_counts, 'negative': negative_counts}

# 결과 데이터프레임 생성
rows = []
for company, data in result.items():
    for type_label, keywords in data.items():
        for keyword, count in keywords:
            rows.append({'Company': company, 'Type': type_label, 'Keyword': keyword, 'Count': count})

df = pd.DataFrame(rows)

# 데이터프레임 출력
df

,Company,Type,Keyword,Count
0,오상헬스케어,positive,기업,212
1,오상헬스케어,positive,상장,179
2,오상헬스케어,positive,진단,145
3,오상헬스케어,positive,오상,134
4,오상헬스케어,positive,바이오,115
...,...,...,...,...
17582,엔에이치스팩30호,positive,증시,6
17583,엔에이치스팩30호,positive,상위,5
17584,엔에이치스팩30호,positive,테마주,5
17585,엔에이치스팩30호,positive,삼성,5


## 데이터베이스에 저장

In [4]:
from mariadb import MariaDB
with open('./aws_config','r',encoding='utf-8') as f:
    db_config = {lines.split('=')[0].strip():lines.split('=')[1].strip() for lines in f.readlines()}
db = MariaDB(db_config)

In [5]:
# 인덱스 초기화
db.truncate('tb_company_keyword')

True

In [6]:
company_detail = list(zip(df['Company'], df['Type'], df['Keyword'], df['Count']))

db.insert_many(
    'tb_company_keyword',
    ','.join(['Company', 'Type', 'Keyword', 'Count']),
    company_detail
)

True